MIT License

Copyright (c) 2021 Taiki Miyagawa and Akinori F. Ebihara

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Train/Valid/Test Splitting (png files) 
DATADIR/UCF101clip50tvt and DATADIR/UCF101clip150tvt will be created.

### UCF101 Stat
- duration = 50
    - Nums of clips in tr:va:te = 1799.8k/50 : 222.7k/50 : 790.35k/50
    - Nums of frames in tr:va:te = 1799.8k : 222.7k : 790.35k
    - Average nums of clips in a class in tr:va:te = 356.3960396,  44.0990099, 156.5049505

- duration = 150
    - Nums of clips in tr:va:te = 2170.05k/150 : 266.25k/150 : 950.55k/150
    - Nums of frames in tr:va:te = 2170.05k : 266.25k : 950.55k
    - Average nums of clips in a class in tr:va:te = 429.71287129 :5.27227723 : 188.22772277

In [2]:
from glob import glob
import os, shutil
from copy import copy
import statistics
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# User-defined params
DATADIR = "Define this first. E.g. /data/t-miyagawa"
duration = 50 # 50 or 150
splitnum = 1 # Official splittng. 1, 2, or 3.

In [3]:
# Paths
newtrdir = "{}/UCF101clip{}tvt/train0{}".format(DATADIR, duration, splitnum) # train images are to be stored here.
newvadir = "{}/UCF101clip{}tvt/valid0{}".format(DATADIR, duration, splitnum) # validation images are to be stored here.
newtedir = "{}/UCF101clip{}tvt/test0{}".format(DATADIR, duration, splitnum) # test images are to be stored here.
allclippaths = sorted(glob("{}/UCF101clip{}/*/*".format(DATADIR, duration))) # all images after clipping.

# Get train/val/test split lists
trtxt = "{}/UCF101/labels/ucfTrainValidTestlist/trainlist0{}.txt".format(DATADIR, splitnum)
vatxt = "{}/UCF101/labels/ucfTrainValidTestlist/validlist0{}.txt".format(DATADIR, splitnum)
tetxt = "{}/UCF101/labels/ucfTrainValidTestlist/testlist0{}.txt".format(DATADIR, splitnum)

if not os.path.exists(newtrdir):
    os.makedirs(newtrdir)
if not os.path.exists(newvadir):
    os.makedirs(newvadir)
if not os.path.exists(newtedir):
    os.makedirs(newtedir)
    
with open(trtxt, mode="r") as f:
    trf = f.readlines()
    trf = [i[:-2] for i in trf]
        
with open(vatxt, mode="r") as f:
    vaf = f.readlines()
    vaf = [i[:-2] for i in vaf]
    
with open(tetxt, mode="r") as f:
    tef = f.readlines()
    tef = [i[:-2] for i in tef]
    
# """
# Returns:
#     newtrdir: A string. Name of the new training dataset directory.
#     newvadir: A string. Name of the new validaiton dataset directory.
#     newtedir: A string. Name of the new test dataset directory.
#     allclippaths: A list of string paths to all clipped files with length = duration.
#         Sorted in alphabetical order.
#         Len = Num of total clips after the re-clipping.
#         E.g., 
#         ['DATADIR/UCF101clip50/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01_cc01',
#          'DATADIR/UCF101clip50/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01_cc01',
#          'DATADIR/UCF101clip50/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01_cc01',
#          'DATADIR/UCF101clip50/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01_cc01',
#          ...
#          'DATADIR/UCF101clip50/YoYo/v_YoYo_g25_c05_cc04',
#          'DATADIR/UCF101clip50/YoYo/v_YoYo_g25_c05_cc04',
#          'DATADIR/UCF101clip50/YoYo/v_YoYo_g25_c05_cc04']
#
#     trf: A list. Len = num of training clips (before re-clipped to length=duration).
#         Therefore, the total num of training data clips with length = duration is more than len(trf).
#         E.g.,
#         ['/ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01',
#          '/ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c02',
#          '/ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03',
#          ...]
#     vaf: A list. Validation set.
#     tef: A list. Test set.
# """

In [4]:
# Get clip paths of tr/va/te 
# It may take a few minutes.
trclippaths = []
vaclippaths = []
teclippaths = []

for _c, v in enumerate(allclippaths):
    # Verbose
    if (_c + 1) % 1000 == 0:
        print("Iter {}/{}".format(_c + 1, len(allclippaths) + 1))
        
    # Initialization for assertion
    _tmptr = 0
    _tmpva = 0
    _tmpte = 0
    
    # Pick out all train images
    for i in trf:
        if i in v:
            trclippaths.append(v)
            _tmptr = 1

    # Pick out all validation images
    for i in vaf:
        if i in v:
            vaclippaths.append(v)
            _tmpva = 1

    # Pick out all test images
    for i in tef:
        if i in v:
            teclippaths.append(v)
            _tmpte = 1
            
    assert _tmptr + _tmpva + _tmpte == 1, "Multi-hit is not impossible in principle: {}, {}, {}".format(_tmptr, _tmpva, _tmpte)

print("Done.")
trclippaths = sorted(trclippaths)
vaclippaths = sorted(vaclippaths)
teclippaths = sorted(teclippaths)

Iter 1000/56258
Iter 2000/56258
Iter 3000/56258
Iter 4000/56258
Iter 5000/56258
Iter 6000/56258
Iter 7000/56258
Iter 8000/56258
Iter 9000/56258
Iter 10000/56258
Iter 11000/56258
Iter 12000/56258
Iter 13000/56258
Iter 14000/56258
Iter 15000/56258
Iter 16000/56258
Iter 17000/56258
Iter 18000/56258
Iter 19000/56258
Iter 20000/56258
Iter 21000/56258
Iter 22000/56258
Iter 23000/56258
Iter 24000/56258
Iter 25000/56258
Iter 26000/56258
Iter 27000/56258
Iter 28000/56258
Iter 29000/56258
Iter 30000/56258
Iter 31000/56258
Iter 32000/56258
Iter 33000/56258
Iter 34000/56258
Iter 35000/56258
Iter 36000/56258
Iter 37000/56258
Iter 38000/56258
Iter 39000/56258
Iter 40000/56258
Iter 41000/56258
Iter 42000/56258
Iter 43000/56258
Iter 44000/56258
Iter 45000/56258
Iter 46000/56258
Iter 47000/56258
Iter 48000/56258
Iter 49000/56258
Iter 50000/56258
Iter 51000/56258
Iter 52000/56258
Iter 53000/56258
Iter 54000/56258
Iter 55000/56258
Iter 56000/56258
Done.


In [5]:
# Get image paths of tr/va/te
trimgpaths = []
vaimgpaths = []
teimgpaths = []

for v in trclippaths:
    trimgpaths.extend(glob(v + "/*.png"))
    
for v in vaclippaths:
    vaimgpaths.extend(glob(v + "/*.png"))
    
for v in teclippaths:
    teimgpaths.extend(glob(v + "/*.png"))
    
trimgpaths = sorted(trimgpaths)
vaimgpaths = sorted(vaimgpaths)
teimgpaths = sorted(teimgpaths)

assert len(trimgpaths) == len(trclippaths) * duration
assert len(vaimgpaths) == len(vaclippaths) * duration
assert len(teimgpaths) == len(teclippaths) * duration

print("Num of frames in tr:va:te = {}k : {}k : {}k".format(len(trimgpaths)/1000, len(vaimgpaths)/1000, len(teimgpaths)/1000))

# """
# Returns:
#     trimgpaths: A list with len = num of all frames in the training dataset.
#         E.g.,
#             ['DATADIR/UCF101clip50/ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01_cc01/0001.png',
#              'DATADIR/UCF101clip50/ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01_cc01/0002.png',
#              'DATADIR/UCF101clip50/ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01_cc01/0003.png',
#                ...]
#     vaimgpaths: A list with len = num of all frames in the validation dataset.
#     teimgpaths: A list with len = num of all frames in the test dataset.
# """

Num of frames in tr:va:te = 1799.8k : 222.7k : 790.35k


## Copy Image Data

In [6]:
# Copy training images
for _c, oldpath in enumerate(trimgpaths):
    if (_c + 1) % 1000 == 0:
        print("Train set iter {} / {}: {}".format(_c + 1, len(trimgpaths), oldpath))
    
    classname = oldpath[oldpath[:oldpath.rfind("/v_")].rfind("/") + 1 :oldpath.rfind("/v_")]
    clipname = oldpath[oldpath.rfind("/v_") + 1 : oldpath.rfind("/")]
    imgname = oldpath[oldpath.rfind("/") + 1 :]
    
    newdir = newtrdir + "/{}/{}".format(classname, clipname)
    if not os.path.exists(newdir):
        os.makedirs(newdir)

    newpath = newdir + "/{}".format(imgname)
    assert newpath[newpath[:newpath[:newpath.rfind("/")].rfind("/")].rfind("/"):] == oldpath[oldpath[:oldpath[:oldpath.rfind("/")].rfind("/")].rfind("/"):],\
    "{}\n{}".format(newpath[newpath[:newpath[:newpath.rfind("/")].rfind("/")].rfind("/"):], oldpath[oldpath[:oldpath[:oldpath.rfind("/")].rfind("/")].rfind("/"):])
    shutil.copy(oldpath, newpath)   

Train set iter 1000 / 1799800: /data/t-miyagawa/UCF101clip50/ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c05_cc06/0050.png
Train set iter 2000 / 1799800: /data/t-miyagawa/UCF101clip50/ApplyEyeMakeup/v_ApplyEyeMakeup_g09_c05_cc01/0050.png
Train set iter 3000 / 1799800: /data/t-miyagawa/UCF101clip50/ApplyEyeMakeup/v_ApplyEyeMakeup_g10_c03_cc01/0050.png
Train set iter 4000 / 1799800: /data/t-miyagawa/UCF101clip50/ApplyEyeMakeup/v_ApplyEyeMakeup_g11_c04_cc02/0050.png
Train set iter 5000 / 1799800: /data/t-miyagawa/UCF101clip50/ApplyEyeMakeup/v_ApplyEyeMakeup_g12_c04_cc04/0050.png
Train set iter 6000 / 1799800: /data/t-miyagawa/UCF101clip50/ApplyEyeMakeup/v_ApplyEyeMakeup_g13_c04_cc01/0050.png
Train set iter 7000 / 1799800: /data/t-miyagawa/UCF101clip50/ApplyEyeMakeup/v_ApplyEyeMakeup_g14_c03_cc04/0050.png
Train set iter 8000 / 1799800: /data/t-miyagawa/UCF101clip50/ApplyEyeMakeup/v_ApplyEyeMakeup_g15_c03_cc04/0050.png
Train set iter 9000 / 1799800: /data/t-miyagawa/UCF101clip50/ApplyEyeMakeup/v_Ap

Train set iter 100000 / 1799800: /data/t-miyagawa/UCF101clip50/BaseballPitch/v_BaseballPitch_g09_c02_cc02/0050.png
Train set iter 101000 / 1799800: /data/t-miyagawa/UCF101clip50/BaseballPitch/v_BaseballPitch_g10_c03_cc03/0050.png
Train set iter 102000 / 1799800: /data/t-miyagawa/UCF101clip50/BaseballPitch/v_BaseballPitch_g12_c01_cc03/0050.png
Train set iter 103000 / 1799800: /data/t-miyagawa/UCF101clip50/BaseballPitch/v_BaseballPitch_g13_c05_cc01/0050.png
Train set iter 104000 / 1799800: /data/t-miyagawa/UCF101clip50/BaseballPitch/v_BaseballPitch_g15_c02_cc01/0050.png
Train set iter 105000 / 1799800: /data/t-miyagawa/UCF101clip50/BaseballPitch/v_BaseballPitch_g16_c04_cc01/0050.png
Train set iter 106000 / 1799800: /data/t-miyagawa/UCF101clip50/BaseballPitch/v_BaseballPitch_g17_c07_cc01/0050.png
Train set iter 107000 / 1799800: /data/t-miyagawa/UCF101clip50/BaseballPitch/v_BaseballPitch_g19_c01_cc01/0050.png
Train set iter 108000 / 1799800: /data/t-miyagawa/UCF101clip50/BaseballPitch/v_B

Train set iter 202000 / 1799800: /data/t-miyagawa/UCF101clip50/BlowDryHair/v_BlowDryHair_g08_c06_cc04/0050.png
Train set iter 203000 / 1799800: /data/t-miyagawa/UCF101clip50/BlowDryHair/v_BlowDryHair_g09_c01_cc06/0050.png
Train set iter 204000 / 1799800: /data/t-miyagawa/UCF101clip50/BlowDryHair/v_BlowDryHair_g09_c03_cc01/0050.png
Train set iter 205000 / 1799800: /data/t-miyagawa/UCF101clip50/BlowDryHair/v_BlowDryHair_g09_c04_cc09/0050.png
Train set iter 206000 / 1799800: /data/t-miyagawa/UCF101clip50/BlowDryHair/v_BlowDryHair_g10_c03_cc05/0050.png
Train set iter 207000 / 1799800: /data/t-miyagawa/UCF101clip50/BlowDryHair/v_BlowDryHair_g10_c07_cc04/0050.png
Train set iter 208000 / 1799800: /data/t-miyagawa/UCF101clip50/BlowDryHair/v_BlowDryHair_g11_c02_cc08/0050.png
Train set iter 209000 / 1799800: /data/t-miyagawa/UCF101clip50/BlowDryHair/v_BlowDryHair_g11_c04_cc10/0050.png
Train set iter 210000 / 1799800: /data/t-miyagawa/UCF101clip50/BlowDryHair/v_BlowDryHair_g11_c07_cc02/0050.png
T

Train set iter 301000 / 1799800: /data/t-miyagawa/UCF101clip50/BoxingSpeedBag/v_BoxingSpeedBag_g20_c01_cc05/0050.png
Train set iter 302000 / 1799800: /data/t-miyagawa/UCF101clip50/BoxingSpeedBag/v_BoxingSpeedBag_g20_c05_cc02/0050.png
Train set iter 303000 / 1799800: /data/t-miyagawa/UCF101clip50/BoxingSpeedBag/v_BoxingSpeedBag_g22_c01_cc02/0050.png
Train set iter 304000 / 1799800: /data/t-miyagawa/UCF101clip50/BoxingSpeedBag/v_BoxingSpeedBag_g23_c01_cc05/0050.png
Train set iter 305000 / 1799800: /data/t-miyagawa/UCF101clip50/BoxingSpeedBag/v_BoxingSpeedBag_g23_c05_cc02/0050.png
Train set iter 306000 / 1799800: /data/t-miyagawa/UCF101clip50/BreastStroke/v_BreastStroke_g08_c04_cc03/0050.png
Train set iter 307000 / 1799800: /data/t-miyagawa/UCF101clip50/BreastStroke/v_BreastStroke_g10_c03_cc01/0050.png
Train set iter 308000 / 1799800: /data/t-miyagawa/UCF101clip50/BreastStroke/v_BreastStroke_g11_c02_cc03/0050.png
Train set iter 309000 / 1799800: /data/t-miyagawa/UCF101clip50/BreastStroke/

Train set iter 396000 / 1799800: /data/t-miyagawa/UCF101clip50/CuttingInKitchen/v_CuttingInKitchen_g16_c04_cc04/0050.png
Train set iter 397000 / 1799800: /data/t-miyagawa/UCF101clip50/CuttingInKitchen/v_CuttingInKitchen_g18_c02_cc02/0050.png
Train set iter 398000 / 1799800: /data/t-miyagawa/UCF101clip50/CuttingInKitchen/v_CuttingInKitchen_g20_c01_cc06/0050.png
Train set iter 399000 / 1799800: /data/t-miyagawa/UCF101clip50/CuttingInKitchen/v_CuttingInKitchen_g21_c01_cc02/0050.png
Train set iter 400000 / 1799800: /data/t-miyagawa/UCF101clip50/CuttingInKitchen/v_CuttingInKitchen_g22_c02_cc02/0050.png
Train set iter 401000 / 1799800: /data/t-miyagawa/UCF101clip50/CuttingInKitchen/v_CuttingInKitchen_g23_c04_cc01/0050.png
Train set iter 402000 / 1799800: /data/t-miyagawa/UCF101clip50/Diving/v_Diving_g08_c06_cc01/0050.png
Train set iter 403000 / 1799800: /data/t-miyagawa/UCF101clip50/Diving/v_Diving_g09_c04_cc03/0050.png
Train set iter 404000 / 1799800: /data/t-miyagawa/UCF101clip50/Diving/v_

...

Train set iter 1407000 / 1799800: /data/t-miyagawa/UCF101clip50/ShavingBeard/v_ShavingBeard_g09_c03_cc02/0050.png
Train set iter 1408000 / 1799800: /data/t-miyagawa/UCF101clip50/ShavingBeard/v_ShavingBeard_g09_c07_cc05/0050.png
Train set iter 1409000 / 1799800: /data/t-miyagawa/UCF101clip50/ShavingBeard/v_ShavingBeard_g10_c04_cc03/0050.png
Train set iter 1410000 / 1799800: /data/t-miyagawa/UCF101clip50/ShavingBeard/v_ShavingBeard_g11_c02_cc02/0050.png
Train set iter 1411000 / 1799800: /data/t-miyagawa/UCF101clip50/ShavingBeard/v_ShavingBeard_g12_c03_cc03/0050.png
Train set iter 1412000 / 1799800: /data/t-miyagawa/UCF101clip50/ShavingBeard/v_ShavingBeard_g13_c02_cc04/0050.png
Train set iter 1413000 / 1799800: /data/t-miyagawa/UCF101clip50/ShavingBeard/v_ShavingBeard_g14_c01_cc04/0050.png
Train set iter 1414000 / 1799800: /data/t-miyagawa/UCF101clip50/ShavingBeard/v_ShavingBeard_g14_c05_cc03/0050.png
Train set iter 1415000 / 1799800: /data/t-miyagawa/UCF101clip50/ShavingBeard/v_ShavingBe

Train set iter 1495000 / 1799800: /data/t-miyagawa/UCF101clip50/SkyDiving/v_SkyDiving_g13_c04_cc03/0050.png
Train set iter 1496000 / 1799800: /data/t-miyagawa/UCF101clip50/SkyDiving/v_SkyDiving_g15_c01_cc02/0050.png
Train set iter 1497000 / 1799800: /data/t-miyagawa/UCF101clip50/SkyDiving/v_SkyDiving_g16_c01_cc03/0050.png
Train set iter 1498000 / 1799800: /data/t-miyagawa/UCF101clip50/SkyDiving/v_SkyDiving_g17_c01_cc04/0050.png
Train set iter 1499000 / 1799800: /data/t-miyagawa/UCF101clip50/SkyDiving/v_SkyDiving_g18_c01_cc03/0050.png
Train set iter 1500000 / 1799800: /data/t-miyagawa/UCF101clip50/SkyDiving/v_SkyDiving_g18_c05_cc05/0050.png
Train set iter 1501000 / 1799800: /data/t-miyagawa/UCF101clip50/SkyDiving/v_SkyDiving_g20_c02_cc01/0050.png
Train set iter 1502000 / 1799800: /data/t-miyagawa/UCF101clip50/SkyDiving/v_SkyDiving_g20_c06_cc02/0050.png
Train set iter 1503000 / 1799800: /data/t-miyagawa/UCF101clip50/SkyDiving/v_SkyDiving_g21_c04_cc02/0050.png
Train set iter 1504000 / 179

Train set iter 1577000 / 1799800: /data/t-miyagawa/UCF101clip50/SumoWrestling/v_SumoWrestling_g22_c02_cc04/0050.png
Train set iter 1578000 / 1799800: /data/t-miyagawa/UCF101clip50/SumoWrestling/v_SumoWrestling_g23_c01_cc05/0050.png
Train set iter 1579000 / 1799800: /data/t-miyagawa/UCF101clip50/Surfing/v_Surfing_g08_c02_cc04/0050.png
Train set iter 1580000 / 1799800: /data/t-miyagawa/UCF101clip50/Surfing/v_Surfing_g08_c05_cc05/0050.png
Train set iter 1581000 / 1799800: /data/t-miyagawa/UCF101clip50/Surfing/v_Surfing_g09_c02_cc03/0050.png
Train set iter 1582000 / 1799800: /data/t-miyagawa/UCF101clip50/Surfing/v_Surfing_g10_c01_cc03/0050.png
Train set iter 1583000 / 1799800: /data/t-miyagawa/UCF101clip50/Surfing/v_Surfing_g10_c05_cc04/0050.png
Train set iter 1584000 / 1799800: /data/t-miyagawa/UCF101clip50/Surfing/v_Surfing_g11_c04_cc01/0050.png
Train set iter 1585000 / 1799800: /data/t-miyagawa/UCF101clip50/Surfing/v_Surfing_g12_c03_cc03/0050.png
Train set iter 1586000 / 1799800: /data/

Train set iter 1662000 / 1799800: /data/t-miyagawa/UCF101clip50/TennisSwing/v_TennisSwing_g22_c03_cc05/0050.png
Train set iter 1663000 / 1799800: /data/t-miyagawa/UCF101clip50/TennisSwing/v_TennisSwing_g23_c01_cc06/0050.png
Train set iter 1664000 / 1799800: /data/t-miyagawa/UCF101clip50/ThrowDiscus/v_ThrowDiscus_g08_c04_cc01/0050.png
Train set iter 1665000 / 1799800: /data/t-miyagawa/UCF101clip50/ThrowDiscus/v_ThrowDiscus_g10_c02_cc03/0050.png
Train set iter 1666000 / 1799800: /data/t-miyagawa/UCF101clip50/ThrowDiscus/v_ThrowDiscus_g11_c05_cc02/0050.png
Train set iter 1667000 / 1799800: /data/t-miyagawa/UCF101clip50/ThrowDiscus/v_ThrowDiscus_g13_c01_cc01/0050.png
Train set iter 1668000 / 1799800: /data/t-miyagawa/UCF101clip50/ThrowDiscus/v_ThrowDiscus_g14_c02_cc02/0050.png
Train set iter 1669000 / 1799800: /data/t-miyagawa/UCF101clip50/ThrowDiscus/v_ThrowDiscus_g16_c02_cc01/0050.png
Train set iter 1670000 / 1799800: /data/t-miyagawa/UCF101clip50/ThrowDiscus/v_ThrowDiscus_g18_c01_cc01/0

Train set iter 1744000 / 1799800: /data/t-miyagawa/UCF101clip50/WalkingWithDog/v_WalkingWithDog_g17_c02_cc01/0050.png
Train set iter 1745000 / 1799800: /data/t-miyagawa/UCF101clip50/WalkingWithDog/v_WalkingWithDog_g18_c01_cc01/0050.png
Train set iter 1746000 / 1799800: /data/t-miyagawa/UCF101clip50/WalkingWithDog/v_WalkingWithDog_g19_c01_cc01/0050.png
Train set iter 1747000 / 1799800: /data/t-miyagawa/UCF101clip50/WalkingWithDog/v_WalkingWithDog_g19_c05_cc01/0050.png
Train set iter 1748000 / 1799800: /data/t-miyagawa/UCF101clip50/WalkingWithDog/v_WalkingWithDog_g20_c04_cc01/0050.png
Train set iter 1749000 / 1799800: /data/t-miyagawa/UCF101clip50/WalkingWithDog/v_WalkingWithDog_g21_c01_cc01/0050.png
Train set iter 1750000 / 1799800: /data/t-miyagawa/UCF101clip50/WalkingWithDog/v_WalkingWithDog_g22_c01_cc01/0050.png
Train set iter 1751000 / 1799800: /data/t-miyagawa/UCF101clip50/WalkingWithDog/v_WalkingWithDog_g23_c01_cc01/0050.png
Train set iter 1752000 / 1799800: /data/t-miyagawa/UCF10

In [6]:
# Copy validation images
for _c, oldpath in enumerate(vaimgpaths):
    if (_c + 1) % 1000 == 0:
        print("Validation set iter {} / {}: {}".format(_c + 1, len(vaimgpaths), oldpath))
    
    classname = oldpath[oldpath[:oldpath.rfind("/v_")].rfind("/") + 1 :oldpath.rfind("/v_")]
    clipname = oldpath[oldpath.rfind("/v_") + 1 : oldpath.rfind("/")]
    imgname = oldpath[oldpath.rfind("/") + 1 :]
    
    newdir = newvadir + "/{}/{}".format(classname, clipname)
    if not os.path.exists(newdir):
        os.makedirs(newdir)

    newpath = newdir + "/{}".format(imgname)
    assert newpath[newpath[:newpath[:newpath.rfind("/")].rfind("/")].rfind("/"):] == oldpath[oldpath[:oldpath[:oldpath.rfind("/")].rfind("/")].rfind("/"):],\
    "{}\n{}".format(newpath[newpath[:newpath[:newpath.rfind("/")].rfind("/")].rfind("/"):], oldpath[oldpath[:oldpath[:oldpath.rfind("/")].rfind("/")].rfind("/"):])
    shutil.copy(oldpath, newpath) 

Validation set iter 1000 / 266250: /data/t-miyagawa/UCF101clip150/ApplyEyeMakeup/v_ApplyEyeMakeup_g24_c05_cc01/0100.png
Validation set iter 2000 / 266250: /data/t-miyagawa/UCF101clip150/ApplyEyeMakeup/v_ApplyEyeMakeup_g25_c01_cc02/0050.png
Validation set iter 3000 / 266250: /data/t-miyagawa/UCF101clip150/ApplyEyeMakeup/v_ApplyEyeMakeup_g25_c06_cc01/0150.png
Validation set iter 4000 / 266250: /data/t-miyagawa/UCF101clip150/ApplyLipstick/v_ApplyLipstick_g24_c05_cc01/0100.png
Validation set iter 5000 / 266250: /data/t-miyagawa/UCF101clip150/ApplyLipstick/v_ApplyLipstick_g25_c03_cc02/0050.png
Validation set iter 6000 / 266250: /data/t-miyagawa/UCF101clip150/Archery/v_Archery_g24_c03_cc01/0150.png
Validation set iter 7000 / 266250: /data/t-miyagawa/UCF101clip150/Archery/v_Archery_g25_c02_cc02/0100.png
Validation set iter 8000 / 266250: /data/t-miyagawa/UCF101clip150/Archery/v_Archery_g25_c06_cc01/0050.png
Validation set iter 9000 / 266250: /data/t-miyagawa/UCF101clip150/BabyCrawling/v_BabyC

Validation set iter 72000 / 266250: /data/t-miyagawa/UCF101clip150/FloorGymnastics/v_FloorGymnastics_g25_c02_cc01/0150.png
Validation set iter 73000 / 266250: /data/t-miyagawa/UCF101clip150/FrisbeeCatch/v_FrisbeeCatch_g24_c03_cc01/0100.png
Validation set iter 74000 / 266250: /data/t-miyagawa/UCF101clip150/FrontCrawl/v_FrontCrawl_g24_c01_cc02/0050.png
Validation set iter 75000 / 266250: /data/t-miyagawa/UCF101clip150/FrontCrawl/v_FrontCrawl_g24_c04_cc02/0150.png
Validation set iter 76000 / 266250: /data/t-miyagawa/UCF101clip150/GolfSwing/v_GolfSwing_g24_c01_cc01/0100.png
Validation set iter 77000 / 266250: /data/t-miyagawa/UCF101clip150/GolfSwing/v_GolfSwing_g24_c05_cc01/0050.png
Validation set iter 78000 / 266250: /data/t-miyagawa/UCF101clip150/GolfSwing/v_GolfSwing_g25_c03_cc01/0150.png
Validation set iter 79000 / 266250: /data/t-miyagawa/UCF101clip150/Haircut/v_Haircut_g24_c01_cc02/0100.png
Validation set iter 80000 / 266250: /data/t-miyagawa/UCF101clip150/Haircut/v_Haircut_g24_c04_c

Validation set iter 145000 / 266250: /data/t-miyagawa/UCF101clip150/PlayingCello/v_PlayingCello_g24_c03_cc01/0100.png
Validation set iter 146000 / 266250: /data/t-miyagawa/UCF101clip150/PlayingCello/v_PlayingCello_g24_c06_cc01/0050.png
Validation set iter 147000 / 266250: /data/t-miyagawa/UCF101clip150/PlayingCello/v_PlayingCello_g25_c01_cc02/0150.png
Validation set iter 148000 / 266250: /data/t-miyagawa/UCF101clip150/PlayingCello/v_PlayingCello_g25_c03_cc03/0100.png
Validation set iter 149000 / 266250: /data/t-miyagawa/UCF101clip150/PlayingCello/v_PlayingCello_g25_c07_cc01/0050.png
Validation set iter 150000 / 266250: /data/t-miyagawa/UCF101clip150/PlayingDaf/v_PlayingDaf_g24_c03_cc01/0150.png
Validation set iter 151000 / 266250: /data/t-miyagawa/UCF101clip150/PlayingDaf/v_PlayingDaf_g25_c02_cc01/0100.png
Validation set iter 152000 / 266250: /data/t-miyagawa/UCF101clip150/PlayingDaf/v_PlayingDaf_g25_c05_cc02/0050.png
Validation set iter 153000 / 266250: /data/t-miyagawa/UCF101clip150/

Validation set iter 216000 / 266250: /data/t-miyagawa/UCF101clip150/Skiing/v_Skiing_g24_c04_cc01/0150.png
Validation set iter 217000 / 266250: /data/t-miyagawa/UCF101clip150/Skiing/v_Skiing_g25_c01_cc02/0100.png
Validation set iter 218000 / 266250: /data/t-miyagawa/UCF101clip150/Skijet/v_Skijet_g24_c01_cc01/0050.png
Validation set iter 219000 / 266250: /data/t-miyagawa/UCF101clip150/Skijet/v_Skijet_g24_c04_cc01/0150.png
Validation set iter 220000 / 266250: /data/t-miyagawa/UCF101clip150/Skijet/v_Skijet_g25_c03_cc01/0100.png
Validation set iter 221000 / 266250: /data/t-miyagawa/UCF101clip150/SkyDiving/v_SkyDiving_g24_c02_cc01/0050.png
Validation set iter 222000 / 266250: /data/t-miyagawa/UCF101clip150/SkyDiving/v_SkyDiving_g25_c01_cc01/0150.png
Validation set iter 223000 / 266250: /data/t-miyagawa/UCF101clip150/SoccerJuggling/v_SoccerJuggling_g24_c01_cc01/0100.png
Validation set iter 224000 / 266250: /data/t-miyagawa/UCF101clip150/SoccerJuggling/v_SoccerJuggling_g24_c04_cc02/0050.png
Va

In [7]:
# Copy test images
for _c, oldpath in enumerate(teimgpaths):
    if (_c + 1) % 1000 == 0:
        print("Test set iter {} / {}: {}".format(_c + 1, len(teimgpaths), oldpath))
    
    classname = oldpath[oldpath[:oldpath.rfind("/v_")].rfind("/") + 1 :oldpath.rfind("/v_")]
    clipname = oldpath[oldpath.rfind("/v_") + 1 : oldpath.rfind("/")]
    imgname = oldpath[oldpath.rfind("/") + 1 :]
    
    newdir = newtedir + "/{}/{}".format(classname, clipname)
    if not os.path.exists(newdir):
        os.makedirs(newdir)

    newpath = newdir + "/{}".format(imgname)
    assert newpath[newpath[:newpath[:newpath.rfind("/")].rfind("/")].rfind("/"):] == oldpath[oldpath[:oldpath[:oldpath.rfind("/")].rfind("/")].rfind("/"):],\
    "{}\n{}".format(newpath[newpath[:newpath[:newpath.rfind("/")].rfind("/")].rfind("/"):], oldpath[oldpath[:oldpath[:oldpath.rfind("/")].rfind("/")].rfind("/"):])
    shutil.copy(oldpath, newpath)   

Test set iter 1000 / 950550: /data/t-miyagawa/UCF101clip150/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04_cc02/0100.png
Test set iter 2000 / 950550: /data/t-miyagawa/UCF101clip150/ApplyEyeMakeup/v_ApplyEyeMakeup_g02_c03_cc01/0050.png
Test set iter 3000 / 950550: /data/t-miyagawa/UCF101clip150/ApplyEyeMakeup/v_ApplyEyeMakeup_g03_c02_cc01/0150.png
Test set iter 4000 / 950550: /data/t-miyagawa/UCF101clip150/ApplyEyeMakeup/v_ApplyEyeMakeup_g04_c02_cc01/0100.png
Test set iter 5000 / 950550: /data/t-miyagawa/UCF101clip150/ApplyEyeMakeup/v_ApplyEyeMakeup_g04_c07_cc01/0050.png
Test set iter 6000 / 950550: /data/t-miyagawa/UCF101clip150/ApplyEyeMakeup/v_ApplyEyeMakeup_g05_c03_cc01/0150.png
Test set iter 7000 / 950550: /data/t-miyagawa/UCF101clip150/ApplyEyeMakeup/v_ApplyEyeMakeup_g05_c07_cc01/0100.png
Test set iter 8000 / 950550: /data/t-miyagawa/UCF101clip150/ApplyEyeMakeup/v_ApplyEyeMakeup_g06_c05_cc01/0050.png
Test set iter 9000 / 950550: /data/t-miyagawa/UCF101clip150/ApplyEyeMakeup/v_ApplyEyeMak

Test set iter 76000 / 950550: /data/t-miyagawa/UCF101clip150/BenchPress/v_BenchPress_g05_c01_cc01/0100.png
Test set iter 77000 / 950550: /data/t-miyagawa/UCF101clip150/BenchPress/v_BenchPress_g05_c06_cc01/0050.png
Test set iter 78000 / 950550: /data/t-miyagawa/UCF101clip150/BenchPress/v_BenchPress_g06_c04_cc01/0150.png
Test set iter 79000 / 950550: /data/t-miyagawa/UCF101clip150/BenchPress/v_BenchPress_g07_c03_cc01/0100.png
Test set iter 80000 / 950550: /data/t-miyagawa/UCF101clip150/Biking/v_Biking_g01_c01_cc02/0050.png
Test set iter 81000 / 950550: /data/t-miyagawa/UCF101clip150/Biking/v_Biking_g01_c04_cc02/0150.png
Test set iter 82000 / 950550: /data/t-miyagawa/UCF101clip150/Biking/v_Biking_g02_c04_cc01/0100.png
Test set iter 83000 / 950550: /data/t-miyagawa/UCF101clip150/Biking/v_Biking_g02_c07_cc02/0050.png
Test set iter 84000 / 950550: /data/t-miyagawa/UCF101clip150/Biking/v_Biking_g03_c02_cc03/0150.png
Test set iter 85000 / 950550: /data/t-miyagawa/UCF101clip150/Biking/v_Biking_

Test set iter 151000 / 950550: /data/t-miyagawa/UCF101clip150/BoxingSpeedBag/v_BoxingSpeedBag_g04_c03_cc02/0100.png
Test set iter 152000 / 950550: /data/t-miyagawa/UCF101clip150/BoxingSpeedBag/v_BoxingSpeedBag_g04_c07_cc01/0050.png
Test set iter 153000 / 950550: /data/t-miyagawa/UCF101clip150/BoxingSpeedBag/v_BoxingSpeedBag_g05_c03_cc02/0150.png
Test set iter 154000 / 950550: /data/t-miyagawa/UCF101clip150/BoxingSpeedBag/v_BoxingSpeedBag_g06_c02_cc02/0100.png
Test set iter 155000 / 950550: /data/t-miyagawa/UCF101clip150/BoxingSpeedBag/v_BoxingSpeedBag_g07_c01_cc02/0050.png
Test set iter 156000 / 950550: /data/t-miyagawa/UCF101clip150/BoxingSpeedBag/v_BoxingSpeedBag_g07_c02_cc04/0150.png
Test set iter 157000 / 950550: /data/t-miyagawa/UCF101clip150/BoxingSpeedBag/v_BoxingSpeedBag_g07_c04_cc03/0100.png
Test set iter 158000 / 950550: /data/t-miyagawa/UCF101clip150/BoxingSpeedBag/v_BoxingSpeedBag_g07_c06_cc02/0050.png
Test set iter 159000 / 950550: /data/t-miyagawa/UCF101clip150/BreastStro

Test set iter 224000 / 950550: /data/t-miyagawa/UCF101clip150/Diving/v_Diving_g06_c03_cc01/0050.png
Test set iter 225000 / 950550: /data/t-miyagawa/UCF101clip150/Diving/v_Diving_g06_c06_cc02/0150.png
Test set iter 226000 / 950550: /data/t-miyagawa/UCF101clip150/Diving/v_Diving_g07_c03_cc01/0100.png
Test set iter 227000 / 950550: /data/t-miyagawa/UCF101clip150/Drumming/v_Drumming_g01_c02_cc02/0050.png
Test set iter 228000 / 950550: /data/t-miyagawa/UCF101clip150/Drumming/v_Drumming_g01_c05_cc02/0150.png
Test set iter 229000 / 950550: /data/t-miyagawa/UCF101clip150/Drumming/v_Drumming_g02_c02_cc01/0100.png
Test set iter 230000 / 950550: /data/t-miyagawa/UCF101clip150/Drumming/v_Drumming_g02_c05_cc02/0050.png
Test set iter 231000 / 950550: /data/t-miyagawa/UCF101clip150/Drumming/v_Drumming_g03_c01_cc02/0150.png
Test set iter 232000 / 950550: /data/t-miyagawa/UCF101clip150/Drumming/v_Drumming_g03_c05_cc02/0100.png
Test set iter 233000 / 950550: /data/t-miyagawa/UCF101clip150/Drumming/v_Dru

Test set iter 300000 / 950550: /data/t-miyagawa/UCF101clip150/HammerThrow/v_HammerThrow_g06_c05_cc01/0150.png
Test set iter 301000 / 950550: /data/t-miyagawa/UCF101clip150/HammerThrow/v_HammerThrow_g07_c03_cc01/0100.png
Test set iter 302000 / 950550: /data/t-miyagawa/UCF101clip150/Hammering/v_Hammering_g02_c01_cc01/0050.png
Test set iter 303000 / 950550: /data/t-miyagawa/UCF101clip150/Hammering/v_Hammering_g03_c02_cc01/0150.png
Test set iter 304000 / 950550: /data/t-miyagawa/UCF101clip150/Hammering/v_Hammering_g04_c03_cc01/0100.png
Test set iter 305000 / 950550: /data/t-miyagawa/UCF101clip150/Hammering/v_Hammering_g06_c01_cc01/0050.png
Test set iter 306000 / 950550: /data/t-miyagawa/UCF101clip150/Hammering/v_Hammering_g07_c01_cc01/0150.png
Test set iter 307000 / 950550: /data/t-miyagawa/UCF101clip150/HandstandPushups/v_HandStandPushups_g01_c03_cc01/0100.png
Test set iter 308000 / 950550: /data/t-miyagawa/UCF101clip150/HandstandPushups/v_HandStandPushups_g03_c02_cc01/0050.png
Test set i

...

Test set iter 602000 / 950550: /data/t-miyagawa/UCF101clip150/PlayingSitar/v_PlayingSitar_g06_c06_cc01/0050.png
Test set iter 603000 / 950550: /data/t-miyagawa/UCF101clip150/PlayingSitar/v_PlayingSitar_g07_c03_cc01/0150.png
Test set iter 604000 / 950550: /data/t-miyagawa/UCF101clip150/PlayingSitar/v_PlayingSitar_g07_c05_cc02/0100.png
Test set iter 605000 / 950550: /data/t-miyagawa/UCF101clip150/PlayingTabla/v_PlayingTabla_g01_c01_cc01/0050.png
Test set iter 606000 / 950550: /data/t-miyagawa/UCF101clip150/PlayingTabla/v_PlayingTabla_g01_c04_cc01/0150.png
Test set iter 607000 / 950550: /data/t-miyagawa/UCF101clip150/PlayingTabla/v_PlayingTabla_g02_c03_cc01/0100.png
Test set iter 608000 / 950550: /data/t-miyagawa/UCF101clip150/PlayingTabla/v_PlayingTabla_g03_c02_cc01/0050.png
Test set iter 609000 / 950550: /data/t-miyagawa/UCF101clip150/PlayingTabla/v_PlayingTabla_g03_c04_cc02/0150.png
Test set iter 610000 / 950550: /data/t-miyagawa/UCF101clip150/PlayingTabla/v_PlayingTabla_g04_c02_cc03/0

Test set iter 679000 / 950550: /data/t-miyagawa/UCF101clip150/RockClimbingIndoor/v_RockClimbingIndoor_g01_c02_cc04/0100.png
Test set iter 680000 / 950550: /data/t-miyagawa/UCF101clip150/RockClimbingIndoor/v_RockClimbingIndoor_g01_c04_cc03/0050.png
Test set iter 681000 / 950550: /data/t-miyagawa/UCF101clip150/RockClimbingIndoor/v_RockClimbingIndoor_g02_c01_cc01/0150.png
Test set iter 682000 / 950550: /data/t-miyagawa/UCF101clip150/RockClimbingIndoor/v_RockClimbingIndoor_g02_c03_cc02/0100.png
Test set iter 683000 / 950550: /data/t-miyagawa/UCF101clip150/RockClimbingIndoor/v_RockClimbingIndoor_g03_c01_cc02/0050.png
Test set iter 684000 / 950550: /data/t-miyagawa/UCF101clip150/RockClimbingIndoor/v_RockClimbingIndoor_g03_c04_cc01/0150.png
Test set iter 685000 / 950550: /data/t-miyagawa/UCF101clip150/RockClimbingIndoor/v_RockClimbingIndoor_g03_c07_cc01/0100.png
Test set iter 686000 / 950550: /data/t-miyagawa/UCF101clip150/RockClimbingIndoor/v_RockClimbingIndoor_g04_c01_cc05/0050.png
Test set

Test set iter 753000 / 950550: /data/t-miyagawa/UCF101clip150/Shotput/v_Shotput_g01_c01_cc01/0150.png
Test set iter 754000 / 950550: /data/t-miyagawa/UCF101clip150/Shotput/v_Shotput_g02_c01_cc01/0100.png
Test set iter 755000 / 950550: /data/t-miyagawa/UCF101clip150/Shotput/v_Shotput_g03_c01_cc01/0050.png
Test set iter 756000 / 950550: /data/t-miyagawa/UCF101clip150/Shotput/v_Shotput_g04_c01_cc01/0150.png
Test set iter 757000 / 950550: /data/t-miyagawa/UCF101clip150/Shotput/v_Shotput_g05_c03_cc01/0100.png
Test set iter 758000 / 950550: /data/t-miyagawa/UCF101clip150/Shotput/v_Shotput_g06_c02_cc01/0050.png
Test set iter 759000 / 950550: /data/t-miyagawa/UCF101clip150/Shotput/v_Shotput_g07_c01_cc01/0150.png
Test set iter 760000 / 950550: /data/t-miyagawa/UCF101clip150/SkateBoarding/v_SkateBoarding_g01_c01_cc01/0100.png
Test set iter 761000 / 950550: /data/t-miyagawa/UCF101clip150/SkateBoarding/v_SkateBoarding_g02_c01_cc01/0050.png
Test set iter 762000 / 950550: /data/t-miyagawa/UCF101clip

Test set iter 829000 / 950550: /data/t-miyagawa/UCF101clip150/SumoWrestling/v_SumoWrestling_g06_c01_cc02/0100.png
Test set iter 830000 / 950550: /data/t-miyagawa/UCF101clip150/SumoWrestling/v_SumoWrestling_g06_c06_cc01/0050.png
Test set iter 831000 / 950550: /data/t-miyagawa/UCF101clip150/SumoWrestling/v_SumoWrestling_g07_c02_cc02/0150.png
Test set iter 832000 / 950550: /data/t-miyagawa/UCF101clip150/SumoWrestling/v_SumoWrestling_g07_c06_cc02/0100.png
Test set iter 833000 / 950550: /data/t-miyagawa/UCF101clip150/Surfing/v_Surfing_g01_c03_cc01/0050.png
Test set iter 834000 / 950550: /data/t-miyagawa/UCF101clip150/Surfing/v_Surfing_g01_c06_cc02/0150.png
Test set iter 835000 / 950550: /data/t-miyagawa/UCF101clip150/Surfing/v_Surfing_g02_c03_cc02/0100.png
Test set iter 836000 / 950550: /data/t-miyagawa/UCF101clip150/Surfing/v_Surfing_g03_c01_cc01/0050.png
Test set iter 837000 / 950550: /data/t-miyagawa/UCF101clip150/Surfing/v_Surfing_g04_c02_cc02/0150.png
Test set iter 838000 / 950550: /da

Test set iter 905000 / 950550: /data/t-miyagawa/UCF101clip150/UnevenBars/v_UnevenBars_g02_c02_cc02/0050.png
Test set iter 906000 / 950550: /data/t-miyagawa/UCF101clip150/UnevenBars/v_UnevenBars_g03_c02_cc01/0150.png
Test set iter 907000 / 950550: /data/t-miyagawa/UCF101clip150/UnevenBars/v_UnevenBars_g04_c04_cc01/0100.png
Test set iter 908000 / 950550: /data/t-miyagawa/UCF101clip150/UnevenBars/v_UnevenBars_g06_c01_cc01/0050.png
Test set iter 909000 / 950550: /data/t-miyagawa/UCF101clip150/UnevenBars/v_UnevenBars_g07_c02_cc01/0150.png
Test set iter 910000 / 950550: /data/t-miyagawa/UCF101clip150/VolleyballSpiking/v_VolleyballSpiking_g01_c03_cc02/0100.png
Test set iter 911000 / 950550: /data/t-miyagawa/UCF101clip150/VolleyballSpiking/v_VolleyballSpiking_g03_c02_cc01/0050.png
Test set iter 912000 / 950550: /data/t-miyagawa/UCF101clip150/VolleyballSpiking/v_VolleyballSpiking_g04_c04_cc01/0150.png
Test set iter 913000 / 950550: /data/t-miyagawa/UCF101clip150/VolleyballSpiking/v_VolleyballSp